# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-28, 09:25:39

   -> Summany DataFrame : 132 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1R000100,Prepay Revenue,2025-03-23,20240528,20250323,"5,870,190,649","10,621,169,006","10,613,408,630","10,613,408,630","5,870,190,357","98,713,171",5,1100,112072
1,20250323,DB2R000100,Postpaid Revenue,2025-03-23,20240528,20250323,"9,295,419,446","16,786,733,611","16,772,782,558","16,772,782,558","9,295,419,031","265,099,860",5,1100,112589
2,20250323,DB2R010100,Postpaid Revenue B2C,2025-03-23,20240528,20250323,"8,517,851,181","15,410,759,305","15,398,600,274","15,398,600,274","8,517,850,766","249,106,903",5,1100,112529
3,20250323,DB2R020100,Postpaid Revenue B2B,2025-03-23,20240528,20250323,"777,568,265","1,375,974,306","1,374,182,284","1,374,182,284","777,568,265","15,992,957",5,1100,73200


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1S000500,Prepay Usage Subs,2025-03-23,20240528,20250323,"1,211,376,361","2,184,083,127","2,182,546,450","2,182,546,450","1,211,376,205","21,245,221",5,1100,113178
1,20250323,DB1S000600,Prepay Revenue Subs,2025-03-23,20240528,20250323,"640,449,011","1,158,930,454","1,158,025,740","1,158,025,740","640,448,954","11,335,618",5,1100,113178
2,20250323,DB2S000500,Postpaid Active Subs,2025-03-23,20240528,20250323,"594,257,292","1,073,519,589","1,072,603,425","1,072,603,425","594,257,149","16,064,172",5,1100,112994
3,20250323,DB2S010500,Postpaid Active Subs B2C,2025-03-23,20240528,20250323,"524,764,054","947,483,511","946,763,870","946,763,870","524,763,911","14,486,262",5,1100,112994
4,20250323,DB2S020400,Postpaid Active Sub B2B,2025-03-23,20240528,20250323,"69,493,238","126,036,078","125,839,555","125,839,555","69,493,238","1,577,910",5,1100,112358


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-28, 09:26:14

   -> Daily DataFrame : 908 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250301


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB1R000100,Prepay Revenue,20250301,2025-03-10,"69,181,547","138,363,094","138,239,253","138,239,253","69,181,547","1,506,675",0,0,5,1098,1123
1,20250323,DB1R000100,Prepay Revenue,20250302,2025-03-11,"70,150,806","140,301,611","140,171,441","140,171,441","70,150,806","1,546,780",0,0,5,1098,1123
2,20250323,DB1R000100,Prepay Revenue,20250303,2025-03-12,"67,382,204","134,764,408","134,648,384","134,648,384","67,382,204","1,449,951",0,0,5,1098,1123
3,20250323,DB1R000100,Prepay Revenue,20250304,2025-03-13,"63,204,663","126,409,327","126,302,821","126,302,821","63,204,663","1,348,670",0,0,5,1098,1123
4,20250323,DB1R000100,Prepay Revenue,20250305,2025-03-14,"63,516,888","127,033,775","126,919,654","126,919,654","63,516,888","1,342,948",0,0,5,1098,1123
5,20250323,DB1R000100,Prepay Revenue,20250306,2025-03-15,"60,419,416","120,838,831","120,734,396","120,734,396","60,419,416","1,279,234",0,0,5,1098,1123
6,20250323,DB1R000100,Prepay Revenue,20250307,2025-03-16,"58,604,342","117,208,684","117,109,408","117,109,408","58,604,342","1,247,824",0,0,5,1098,1123
7,20250323,DB1R000100,Prepay Revenue,20250308,2025-03-17,"57,566,325","115,132,650","115,037,718","115,037,718","57,566,325","1,248,182",0,0,5,1098,1123
8,20250323,DB1R000100,Prepay Revenue,20250309,2025-03-18,"55,706,984","111,413,967","111,326,029","111,326,029","55,706,984","1,225,405",0,0,5,1098,1123
9,20250323,DB1R000100,Prepay Revenue,20250310,2025-03-19,"57,966,061","115,932,122","115,836,760","115,836,760","57,966,061","1,149,640",0,0,5,1098,1123


In [7]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250301


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R000100,Postpaid Revenue,20250301,2025-03-10,"6,260,862","12,521,724","12,513,360","12,513,360","6,260,862","234,917",0,0,5,1097,1113
1,20250323,DB2R000100,Postpaid Revenue,20250302,2025-03-11,"234,603,697","469,207,394","468,448,085","468,448,085","234,603,697","8,883,105",0,0,5,1098,1118
2,20250323,DB2R000100,Postpaid Revenue,20250303,2025-03-12,"90,427,677","180,855,354","180,713,324","180,713,324","90,427,677","3,205,434",0,0,5,1098,1117
3,20250323,DB2R000100,Postpaid Revenue,20250304,2025-03-13,"91,485,050","182,970,099","182,831,314","182,831,314","91,485,050","3,173,583",0,0,5,1097,1114
4,20250323,DB2R000100,Postpaid Revenue,20250305,2025-03-14,"86,943,936","173,887,872","173,764,026","173,764,026","86,943,936","3,102,551",0,0,5,1097,1115
5,20250323,DB2R000100,Postpaid Revenue,20250306,2025-03-15,"90,781,279","181,562,558","181,392,108","181,392,108","90,781,279","3,345,782",0,0,5,1097,1114
6,20250323,DB2R000100,Postpaid Revenue,20250307,2025-03-16,"87,524,440","175,048,880","174,891,142","174,891,142","87,524,440","3,019,365",0,0,5,1098,1119
7,20250323,DB2R000100,Postpaid Revenue,20250308,2025-03-17,"87,408,884","174,817,768","174,667,759","174,667,759","87,408,884","3,089,710",0,0,5,1098,1114
8,20250323,DB2R000100,Postpaid Revenue,20250309,2025-03-18,"84,807,154","169,614,307","169,470,995","169,470,995","84,807,154","2,985,060",0,0,5,1098,1119
9,20250323,DB2R000100,Postpaid Revenue,20250310,2025-03-19,"92,896,558","185,793,116","185,638,602","185,638,602","92,896,558","3,155,328",0,0,5,1097,1115


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250301


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R010100,Postpaid Revenue B2C,20250301,2025-03-10,"4,852,098","9,704,195","9,696,467","9,696,467","4,852,098","225,659",0,0,5,1096,1111
1,20250323,DB2R010100,Postpaid Revenue B2C,20250302,2025-03-11,"96,478,080","192,956,159","192,783,387","192,783,387","96,478,080","3,754,454",0,0,5,1098,1117
2,20250323,DB2R010100,Postpaid Revenue B2C,20250303,2025-03-12,"87,959,314","175,918,628","175,776,914","175,776,914","87,959,314","3,199,187",0,0,5,1098,1117
3,20250323,DB2R010100,Postpaid Revenue B2C,20250304,2025-03-13,"87,867,555","175,735,110","175,597,473","175,597,473","87,867,555","3,163,458",0,0,5,1097,1114
4,20250323,DB2R010100,Postpaid Revenue B2C,20250305,2025-03-14,"85,349,305","170,698,611","170,575,067","170,575,067","85,349,305","3,092,071",0,0,5,1097,1115
5,20250323,DB2R010100,Postpaid Revenue B2C,20250306,2025-03-15,"89,507,469","179,014,938","178,845,934","178,845,934","89,507,469","3,327,986",0,0,5,1097,1113
6,20250323,DB2R010100,Postpaid Revenue B2C,20250307,2025-03-16,"86,258,061","172,516,121","172,358,553","172,358,553","86,258,061","3,015,433",0,0,5,1098,1118
7,20250323,DB2R010100,Postpaid Revenue B2C,20250308,2025-03-17,"84,570,570","169,141,141","168,991,425","168,991,425","84,570,570","3,086,829",0,0,5,1098,1113
8,20250323,DB2R010100,Postpaid Revenue B2C,20250309,2025-03-18,"83,952,325","167,904,649","167,761,257","167,761,257","83,952,325","2,978,836",0,0,5,1098,1118
9,20250323,DB2R010100,Postpaid Revenue B2C,20250310,2025-03-19,"87,349,442","174,698,884","174,544,188","174,544,188","87,349,442","3,150,245",0,0,5,1097,1114


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250301


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250323,DB2R020100,Postpaid Revenue B2B,20250301,2025-03-10,"1,408,764","2,817,528","2,816,894","2,816,894","1,408,764","9,258",0,0,5,826,841
1,20250323,DB2R020100,Postpaid Revenue B2B,20250302,2025-03-11,"138,125,618","276,251,235","275,664,698","275,664,698","138,125,618","5,128,651",0,0,5,1098,1114
2,20250323,DB2R020100,Postpaid Revenue B2B,20250303,2025-03-12,"2,468,363","4,936,726","4,936,410","4,936,410","2,468,363","6,247",0,0,5,627,642
3,20250323,DB2R020100,Postpaid Revenue B2B,20250304,2025-03-13,"3,617,495","7,234,990","7,233,841","7,233,841","3,617,495","10,125",0,0,5,596,611
4,20250323,DB2R020100,Postpaid Revenue B2B,20250305,2025-03-14,"1,594,631","3,189,261","3,188,960","3,188,960","1,594,631","10,480",0,0,5,593,608
5,20250323,DB2R020100,Postpaid Revenue B2B,20250306,2025-03-15,"1,273,810","2,547,621","2,546,174","2,546,174","1,273,810","17,796",0,0,5,737,752
6,20250323,DB2R020100,Postpaid Revenue B2B,20250307,2025-03-16,"1,266,380","2,532,759","2,532,589","2,532,589","1,266,380","3,932",0,0,5,611,626
7,20250323,DB2R020100,Postpaid Revenue B2B,20250308,2025-03-17,"2,838,313","5,676,627","5,676,334","5,676,334","2,838,313","2,881",0,0,5,566,581
8,20250323,DB2R020100,Postpaid Revenue B2B,20250309,2025-03-18,"854,829","1,709,658","1,709,738","1,709,738","854,829","6,224",0,0,5,565,580
9,20250323,DB2R020100,Postpaid Revenue B2B,20250310,2025-03-19,"5,547,116","11,094,232","11,094,414","11,094,414","5,547,116","5,082",0,0,5,635,650


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]